# India rural habitations list

Coordinate location of rural habitations or settlements in India with geographic codes.

Sources:
- `CC-by/India-GODL` PMGSY Rural Facilities Dataset, Ministry of Rural Development, 2021 http://omms.nic.in
- `CC-by/India-GODL` India Local Government Directory, Ministry of Panchayati Raj, 2021 https://lgdirectory.gov.in/downloadDirectory.do

Sample:
```
habitation_name, habitation_code, block_name, block_code, subdistrict, subdistrict_code, district, district_code, state, state_code
```

## Habitations

The village (gaon) has developed three main meanings in India: (1) the GP unit of local government; (2) the revenue village that is important in Census data collection and mapping; and (3) the habitation which is the smallest settlement for project planning purposes. A GP may include multiple villages, each of which may have multiple habitations. 

Habitations are the smallest level of village settlement that commonly have 10s–100s of households. As noted above, each gram panchayat has an average of 6.6 habitations.

![](https://iwa.silverchair-cdn.com/iwa/content_public/journal/washdev/9/3/10.2166_washdev.2019.196/4/m_washdev-d-18-00196f01.png?Expires=1619101968&Signature=2M3XOOtz3uD-LCPfT9yRQsVhTmlSSYNN3k7MXdMmZPHk4~METZjcdanSsCt8r5j154dhOtyb7Ric-BQQlFIUhnC4kzQr~cIH1AMCuuLyIBYSfJqm5PsxRwCEFp2Kr34L2PAUG3u21yufVBmahAJc87rQf-fuUcs~MZ-a8tbh3WxhyyHEOxhwvXcRcfeJGk49lKQhyqNC79AXubErypXpQEz5BIuUGhpr5CXMFQRWNpBx3LMyiWSpw2pzP9XjI2nkVUb2q9T1Gmp~PYGRRhkhqCfB9EZXCnQDRTe1wYHstDcZnHRwNAH1pjVjNVVeoHkOsHIY~CgAOfunpkRPuzh4Nw__&Key-Pair-Id=APKAIE5G5CRDK6RD3PGA)

Source: [Habitations, villages, and gram panchayats: local drinking water planning in rural India with a Pune district case study, 2019](https://iwaponline.com/washdev/article/9/3/522/68671/Habitations-villages-and-gram-panchayats-local)

### Read sources

In [ ]:
import pandas as pd
import numpy as np

rural_facilities = pd.read_csv("../input/770k-geotagged-rural-facilities-in-india-pmgsy/pmgsy__india_facilities.csv")

### Import directory of block level LGD codes

- Index using a `block-slug` composed of a canonical representation of the administrative heirarchy

In [ ]:
lgd_blocks = pd.read_csv("../input/india-local-government-directory/statistical/blocks.csv")

lgd_blocks.columns = ['s.no','state_code', 'state_name','district_code','district_name','block_code','block_version','block_name']
lgd_blocks = lgd_blocks[['state_code', 'state_name','district_code','district_name','block_code','block_name']]

lgd_blocks['block_key'] = lgd_blocks['state_name']+'-'+lgd_blocks_src['district_name']+'-'+lgd_blocks_src['block_name']
lgd_blocks['block_key'] = lgd_blocks['block_key'].str.lower()
lgd_blocks['block_key'] = lgd_blocks['block_key'].str.replace(r"[^a-zA-Z0-9\-]", '')
lgd_blocks = lgd_blocks.set_index('block_key')
lgd_blocks



### Import mapping of block names in the dataset to names in the LGD block directory
- Index using a lookup key

In [ ]:
block_key_map = pd.read_csv("../input/india-pmgsy-block-codes/PMGSY block code lookup - pmgsy-blocks.csv")
block_key_map = block_key_map[['block-lookup','pmgsy-block-key']]

block_key_map.columns = ['lgd_block_key','data_block_key']

block_key_map['lgd_block_key'] = block_key_map['lgd_block_key'].str.replace(r"[^a-zA-Z0-9\-]", '')
block_key_map['data_block_key'] = block_key_map['data_block_key'].str.replace(r"[^a-zA-Z0-9\-]", '')

block_key_map = block_key_map.set_index('data_block_key')

block_key_map.head()

In [ ]:
block_lookup.join(lgd_blocks, lsuffix='_caller', rsuffix='_other')

In [ ]:
rural_facilities.head()

### Create a list of habitations
- Group rural facility locations by habitation
- Create a `block-slug` that will help add lgd codes upto block level

In [ ]:
rural_habitations = rural_facilities.groupby('Habitation ID').first()
rural_habitations=rural_habitations[['State','District','Block','Habitation Name','Lattitude','Longitude']]

In [ ]:
rural_habitations['block_key'] = rural_habitations['State']+'-'+rural_habitations['District']+'-'+rural_habitations['Block']
rural_habitations['block_key'] = rural_habitations['block_key'].str.replace(r"[^a-zA-Z0-9\-]", '')
rural_habitations['block_key'] = rural_habitations['block_key'].str.lower()
rural_habitations

In [ ]:
habitation_blocks_lgd = rural_habitations.set_index('block_key').join(lgd_blocks[['block_name','block_code']])
habitation_blocks_lgd_unmapped = habitation_blocks_lgd[habitation_blocks_lgd['block_code'].isnull()]

habitation_blocks_lgd_unmapped

In [ ]:
habitation_blocks_lgd_unmapped = habitation_blocks_lgd_unmapped.join(block_key_map)
# habitation_blocks_lgd_unmapped = habitation_blocks_lgd_unmapped.set_index('lgd_block_key').join(lgd_blocks[['block_name','block_code']], lsuffix='_caller', rsuffix='_other')
# habitation_blocks_lgd_unmapped = habitation_blocks_lgd[habitation_blocks_lgd['block_code'].isnull()]

habitation_blocks_lgd_unmapped

In [ ]:
rural_habitations_lgd = rural_habitations_lgd.set_index('block-lookup').join(lgd_blocks.set_index(['block-slug']))

In [ ]:
rural_habitations_lgd.to_csv('/kaggle/working/df.csv')

In [ ]:
rural_habitations_lgd['Block Code'].isnull().count()

In [ ]:
rural_habitations_lgd